<a href="https://colab.research.google.com/github/Idriss-jedid/Automated-Contract-Photo-Verification/blob/master/Computer%20Vision%20/%20Car_Condition_Classification_Complet_vs_Noncomplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from sklearn.metrics import f1_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/data_car_direction.csv'


In [ ]:
# Load the CSV file
df = pd.read_csv(data_path)

In [ ]:
class_names = df['class'].unique()
label_map = {name: idx for idx, name in enumerate(class_names)}
df['class'] = df['class'].map(label_map)

In [ ]:
# Define image directory
image_dir = '/content/drive/MyDrive/filtered_data'


In [ ]:
# Custom Dataset class
class ImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label



In [ ]:
# Split the data into train, validation, and test sets  [0.6 0.2 0.2]
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df['class'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['class'])

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create datasets
train_dataset = ImageDataset(train_df, image_dir, transform=transform)
val_dataset = ImageDataset(val_df, image_dir, transform=transform)
test_dataset = ImageDataset(test_df, image_dir, transform=transform)

In [ ]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]


In [ ]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_one_epoch(model, criterion, optimizer, data_loader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [ ]:
def evaluate_model(model, criterion, data_loader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    epoch_f1 = f1_score(all_labels, all_preds, average='weighted')
    return epoch_loss, epoch_acc, epoch_f1


In [ ]:
num_epochs=20
best_model_wts = None
best_loss = float('inf')
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, criterion, optimizer, train_loader, device)
    val_loss, val_acc, val_f1 = evaluate_model(model, criterion, val_loader, device)

    print(f'Epoch {epoch}/{num_epochs - 1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}')

    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = model.state_dict()
        torch.save(model.state_dict(), 'best_model.pth')

Epoch 0/19, Train Loss: 0.4468, Train Accuracy: 0.8533
Val Loss: 0.4147, Val Accuracy: 0.8447, F1 Score: 0.8510
Epoch 1/19, Train Loss: 0.2066, Train Accuracy: 0.9319
Val Loss: 0.2818, Val Accuracy: 0.9036, F1 Score: 0.9006
Epoch 2/19, Train Loss: 0.1528, Train Accuracy: 0.9440
Val Loss: 0.3898, Val Accuracy: 0.8787, F1 Score: 0.8840
Epoch 3/19, Train Loss: 0.1572, Train Accuracy: 0.9437
Val Loss: 0.3475, Val Accuracy: 0.8832, F1 Score: 0.8817
Epoch 4/19, Train Loss: 0.0775, Train Accuracy: 0.9705
Val Loss: 0.3060, Val Accuracy: 0.9138, F1 Score: 0.9121
Epoch 5/19, Train Loss: 0.0839, Train Accuracy: 0.9716
Val Loss: 0.5680, Val Accuracy: 0.8277, F1 Score: 0.8376
Epoch 6/19, Train Loss: 0.0595, Train Accuracy: 0.9819
Val Loss: 0.3462, Val Accuracy: 0.9116, F1 Score: 0.9108
Epoch 7/19, Train Loss: 0.0527, Train Accuracy: 0.9830
Val Loss: 0.4056, Val Accuracy: 0.9093, F1 Score: 0.9075
Epoch 8/19, Train Loss: 0.0379, Train Accuracy: 0.9887
Val Loss: 0.2961, Val Accuracy: 0.9184, F1 Score:

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))


<ipython-input-18-97ca0473330f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


<All keys matched successfully>

In [ ]:
# Evaluate on test set
model.eval()
all_preds = []
all_labels = []
total_loss = 0
correct = 0
total = 0

criterion = nn.CrossEntropyLoss()  # Make sure to import nn from torch

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Calculate loss
        loss = criterion(outputs, labels)
        total_loss += loss.item() * inputs.size(0)

        # Calculate accuracy
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate metrics
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = correct / total
avg_loss = total_loss / total

print(f'Test Weighted F1 Score: {f1:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Loss: {avg_loss:.4f}')

Test Weighted F1 Score: 0.9338
Test Accuracy: 0.9342
Test Loss: 0.2581


In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 38.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import onnx

# Your class names
class_names = ['autre', 'avant', 'arrier', 'droite', 'gauche']

# Load pre-trained ResNet-18 model and modify the final layer
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # Adjusting the output layer to match the number of classes

# Load the state dictionary from your trained model
state_dict = torch.load("best_model.pth", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "modelresnet.onnx", input_names=['input'], output_names=['class_scores'])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-22-18bd9951839e>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detai